Find Active users that logged in to their accounts for 5 or more consecutive days.

In [18]:
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery_storage
import pandas as pd
import os

In [19]:
credentials = service_account.Credentials.from_service_account_file("/Users/al/Downloads/bq.json")
project_id = 'ga-data-1250'
bqclient = bigquery.Client(credentials=credentials, project=project_id)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

In [20]:
query = """
SELECT DISTINCT
        fullVisitorId AS user_id, 
        DATE(TIMESTAMP_SECONDS(visitStartTime)) AS visit_day
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201701*`
LIMIT 10
"""

results = (
    bqclient.query(query)
            .result()
            .to_dataframe(
                bqstorage_client=bqstorageclient)
)


In [21]:
results

,user_id,visit_day
0,0766527698986579906,2017-01-24
1,6961365578873598870,2017-01-24
2,9324540754975564389,2017-01-24
3,4270470652366319162,2017-01-24
4,8472838377035107933,2017-01-25
5,4187864778112777591,2017-01-24
6,0744438076926814772,2017-01-25
7,7541036483940304988,2017-01-25
8,9620460454484605087,2017-01-24
9,4717688672026497695,2017-01-25


In [23]:
query = """
WITH 
raw AS (
    SELECT DISTINCT
            fullVisitorId AS user_id, 
            DATE(TIMESTAMP_SECONDS(visitStartTime)) AS visit_day,
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_2017011*`
),
visits AS (
    SELECT  *,
            ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY visit_day) as rn
    FROM raw
)

SELECT user_id, 
         COUNT(*) as num_days
FROM visits
GROUP BY user_id, DATE_ADD(visit_day, INTERVAL -rn DAY)
HAVING COUNT(*) >= 7
"""

results = (
    bqclient.query(query)
            .result()
            .to_dataframe(
                bqstorage_client=bqstorageclient)
)

results

,user_id,num_days
0,6521210701805859401,10
